<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Znz3OTZ6zCbdNcnrJqVwhBxY0NzNvcZj?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions

Learn by building. Get expert mentorship and work on real AI projects.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




# Price Comparison Application Using ```gpt-oss-120b``` And CrewAI

This notebook demonstrates how to build a price comparison application using **CrewAI** and the brand new **gpt-oss-120b**.
This will search for the best deals on a specified product across multiple e-commerce websites and compare the prices to find the best deal.

 You will need :  
 - A Cerebras API key, which you can obtain from the [Cerebras Inference QuickStart Guide](https://inference-docs.cerebras.ai/quickstart),
 - A Serper API key, which you can get from the [Serper API website](https://serper.dev/).

It utilizes the serper.dev API to fetch and display the most relevant search results based on the query provided by the user.

##Installation

In [ ]:
!pip install crewai crewai_tools

##Imports

In [ ]:
from crewai import LLM
from google.colab import userdata
import os
from crewai import LLM
from crewai import Agent, Crew, Task, Process
from crewai_tools import SerperDevTool

## Configure ```gpt-oss-120b``` And CrewAI

In [ ]:
api_key = userdata.get('CEREBRAS_API_KEY')
if isinstance(api_key, list):
    api_key = api_key[0]
# Initialize the LLM with the cerebras/gpt-oss-120b
cllm = LLM(
    model="cerebras/gpt-oss-120b",
    api_key=api_key,
    base_url="https://api.cerebras.ai/v1"
)



## Initializing Serper API

In [ ]:
api_key= userdata.get('SERPER_API_KEY')
if isinstance(api_key, list):
    api_key = api_key[0]
os.environ['SERPER_API_KEY'] = api_key
# Initialize the tool for internet searching capabilities
search_tool = SerperDevTool(country="in")

## Setting Up Agents

In [ ]:
# Create a price researcher agent
price_researcher = Agent(
    role='Price Researcher',
    llm = cllm,
    goal='Find the best deals on {topic} from popular e-commerce websites like amazon , myntra , flipkart , ebay ',
    verbose=True,
    memory=True,
    backstory=(
        "Expert in finding the best deals on various e-commerce websites."
    ),
    tools=[search_tool],
    allow_delegation=True,

)

# Create a price comparison agent
price_comparator = Agent(
    role='Price Comparator',
    llm = cllm,
    goal='Compare prices from different websites and find the best deal.',
    verbose=True,
    memory=True,
    backstory=(
        "Expert in comparing prices and finding the best deals."
    ),
    tools=[search_tool],
    allow_delegation=False,


)


## Configure Tasks

In [ ]:
# Define the research task
research_task = Task(
    description=(
        "Search for the best deals on {topic} from popular e-commerce websites"
    ),
    expected_output='A dictionary with prices from different websites.',
    tools=[search_tool],
    agent=price_researcher,
)

# Define the comparison task
comparison_task = Task(
    description=(
        "Compare prices from different websites and find the best deal."
    ),
    expected_output='The best deal with the lowest price.',
    tools=[search_tool],
    agent=price_comparator,
    async_execution=False,
      )


## Forming The Price Comparison Crew

In [ ]:
crew = Crew(
    agents=[price_researcher, price_comparator],
    tasks=[research_task, comparison_task],
    process=Process.sequential
    )
# launch the agents' crew
result = crew.kickoff(inputs={'topic': 'red t-shirt'})

# Print the result
print("Best Deal Found:")
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Price Researcher                                                                                        │
│                                                                                                                 │
│  Task: Search for the best deals on red t-shirt from popular e-commerce websites                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Price Researcher                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "action": "Search the internet with Serper",                                                                 │
│    "action_input": {                                                                                            │
│      "search_query": "red t-shirt site:amazon.com"                                                              │
│    }                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Price Comparator                                                                                        │
│                                                                                                                 │
│  Task: Compare prices from different websites and find the best deal.                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Price Comparator                                                                                        │
│                                                                                                                 │
│  Thought: We need to output in required format.Thought: I need to find the current price of a red t‑shirt on    │
│  Amazon, then compare it with prices on other major retailers such as eBay and Walmart to determine the lowest  │
│  price.                                                                                                         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"red t\\u2011shirt site:amazon.com\"}"                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Price Comparator                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I have found a plain red T‑shirt on Amazon listed at **$10.26** (see the “Plain Red          │
│  T‑shirt” search snippet). On eBay, a plain red T‑shirt listing shows a price of **$2.25** (see the “Eddie      │
│  Bauer T‑Shirt Women’s Size XL Plain Red … $2.25$2.25.” snippet). To complete the comparison, I need a          │
│  comparable price from Walmart.                                                                                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"\\\"plain red t\\u2011shirt\\\" site:walmart.com\"}"                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': '"plain red t‑shirt" site:walmart.com', 'type': 'search', 'gl': 'in', 'num': 10,    │
│  'engine': 'google'}, 'organic': [{'title': 'Fznquz Plain Shirts Men V Neck Short Sleeve Red T ...', 'link':    │
│  'https://business.walmart.com/ip/Fznquz-Plain-Shirts-Men-V-Neck-Short-Sleeve-Red-T-Shirt-Casual-Classic-Men-s  │
│  -Shirts-XXL/15469118168', 'snippet': '... Casual Men s Shirt XXXLFznquz V Neck Men Big and Tall Shirts Short   │
│  Sleeve Plain Red T Shirt Casual Classic Tees XXXL. Color:Wine. $5.25. $5.25. $5.25. $5.25.', 'position': 1},   │
│  {'title': 'Fznquz V Neck Men Big and Tall Shirts Short Sleeve Plain Red T ...', 'link':                        │
│  'https://business.walmart.com/ip/Fznquz-V-Neck-Men-Big-and-Tall-Shirts-Short-Sleeve-Plain-Red-T-Shirt-Casual-  │
│  Classic-Tees-XXXL/15456154028', 'snippet': 'Fznquz V Neck Men Big and Tall Shirts Short Sleeve Plain Red T     │
│  Shirt Casual Classic Tees XXXL. USDNow $5.66. was $6.99$6.99. Prices may vary online, ...', 'position': 2},    │
│  {'title': 'Kids T Shirts Plain Red T Shirt Kids Pink Long Sleeve ...', 'link':                                 │
│  'https://www.walmart.com/ip/Kids-T-Shirts-Plain-Red-T-Shirt-Kids-Pink-Long-Sleeve-Toddler-Shirt-Toddler-Boys-  │
│  Clothes-Clearance-Sale-White-T-Shirts-for-Toddler-Boys/14720002460', 'snippet': 'Kids T Shirts Plain Red T     │
│  Shirt Kids Pink Long Sleeve Toddler Shirt Toddler Boys Clothes Clearance Sale White T Shirts for Toddler       │
│  Boys. Free 30-day returns.', 'position': 3}, {'title': 'Children T Shirts Plain Red T Shirt Kids Funny         │
│  Toddler Shirts ...', 'link':                                                                                   │
│  'https://business.walmart.com/ip/Children-T-Shirts-Plain-Red-T-Shirt-Kids-Funny-Toddler-Shirts-Kids-Shirts-Cl  │
│  earance-Toddler-Waffle-Shirt/15189554177', 'snippet': 'Children T Shirts Plain Red T Shirt Kids Funny Toddler  │
│  Shirts Kids Shirts Clearance Toddler Waffle Shirt ; Color:Hot Pink ; Clothing Size:3T. Size guide ; Arrives    │
│  ...', 'position': 4}, {'title': 'Kids White Tee Shirts Plain Red T Shirt Kids Toddler Girl Dress ...',         │
│  'link': 'https://www.walmart.com/ip/Kids-White-Tee-Shirts-Plain-Red-T-Shirt-Kids...                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Price Comparator                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Thought: I have gathered the following price information for a comparable plain red T‑shirt:                   │
│                                                                                                                 │
│  | Site   | Product (as listed in the search snippet)                              | Price (USD) | Link |       │
│  |--------|-------------------------------------------------------------------------|------------|------|       │
│  | Amazon | Plain Red T‑shirt (multipack)                                           | $10.26 |                  │
│  https://www.amazon.com/plain-red-t-shirt/s?k=plain+red+t-shirt |                                               │
│  | eBay   | Eddie Bauer T‑Shirt Women’s Size XL Plain Red T‑Shirt (price shown twice) | $2.25 |                 │
│  https://www.ebay.com/itm/116347904413 |                                                                        │
│  | Walmart| Fznquz Plain Shirts Men V Neck Short Sleeve Red T‑Shirt (Casual Classic) | $5.25 |                  │
│  https://business.walmart.com/ip/Fznquz-Plain-Shirts-Men-V-Neck-Short-Sleeve-Red-T-Shirt-Casual-Classic-Men-s-  │
│  Shirts-XXL/15469118168 |                                                                                       │
│                                                                                                                 │
│  The lowest price among these listings is the eBay listing at **$2.25**.                                        │
│                                                                                                                 │
│  **Best Deal (lowest price):**                                                                                  │
│                                                                                                                 │
│  - **Website:** eBay                                                                                            │
│  - **Product:** Eddie Bauer T‑Shirt Women’s Size XL Plain Red T‑Shirt                                           │
│  - **Price:** **$2.25**                                                                                         │
│  - **Link:** https://www.ebay.com/itm/116347904413                                                              │
│                                                                                                                 │
│  This eBay listing offers the cheapest plain red T‑shirt compared to the Amazon and Walmart alternatives.       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Best Deal Found:
Thought: I have gathered the following price information for a comparable plain red T‑shirt:

| Site   | Product (as listed in the search snippet)                              | Price (USD) | Link |
|--------|-------------------------------------------------------------------------|------------|------|
| Amazon | Plain Red T‑shirt (multipack)                                           | $10.26 | https://www.amazon.com/plain-red-t-shirt/s?k=plain+red+t-shirt |
| eBay   | Eddie Bauer T‑Shirt Women’s Size XL Plain Red T‑Shirt (price shown twice) | $2.25 | https://www.ebay.com/itm/116347904413 |
| Walmart| Fznquz Plain Shirts Men V Neck Short Sleeve Red T‑Shirt (Casual Classic) | $5.25 | https://business.walmart.com/ip/Fznquz-Plain-Shirts-Men-V-Neck-Short-Sleeve-Red-T-Shirt-Casual-Classic-Men-s-Shirts-XXL/15469118168 |

The lowest price among these listings is the eBay listing at **$2.25**.

**Best Deal (lowest price):**

- **Website:** eBay  
- **Product:** Eddie Bauer T‑S